In [1]:
# required Python imports
import numpy as np
import matplotlib.pyplot as plt
# from matplotlib import cm
from types import SimpleNamespace

import ftir_funct as f

module FTIR v.2024.2.22 imported


## Step 1: Create the database

In [2]:
# STEP 1.1: generate Euler angles and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=45))
database.euler.shape

(45, 3)

In [3]:
# STEP 1.2: Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(90, 50, 20), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

In [4]:
# STEP 1.3: Generate 16 transmission and azimuth angles pairs for each Euler angle
angles = np.arange(0, 360, 22.5) 

# initialize variables
T_vals = []
azi_vals = []

for euler in database.euler:
    # rotate
    x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

    # extract XY intersection
    xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

    # get the indexes of specific angles
    indexes = f.find_nearest(xy_vectors['angles'], angles)

    # append values
    T_vals.append(xy_vectors.loc[indexes, ['T']].T.values.tolist()[0])
    azi_vals.append(xy_vectors.loc[indexes, ['angles']].T.values.tolist()[0])

# store values in database
database.T_values = np.array(T_vals)
database.azimuths = np.array(azi_vals)

In [5]:
np.arange(0, 360, 22.5)

array([  0. ,  22.5,  45. ,  67.5,  90. , 112.5, 135. , 157.5, 180. ,
       202.5, 225. , 247.5, 270. , 292.5, 315. , 337.5])

In [6]:
print(database.euler.shape)
print(database.T_values.shape)
print(database.azimuths.shape)

(45, 3)
(45, 16)
(45, 16)


In [7]:
for index, orientation in enumerate(database.euler):
    print('Real:', orientation)

    measures = np.column_stack((database.T_values[index],
                                database.azimuths[index],
                                np.full_like(database.azimuths[index], 90)))

    f.find_orientation(measurements=measures,
                       params=(90, 50, 20),
                       num_guesses=80)
    print('----------------------')

Real: [0 0 0]
Calculated orientation: [ 42.   0. 138.]
----------------------
Real: [ 0  0 45]
Calculated orientation: [36.  0.  9.]
----------------------
Real: [ 0  0 90]
Calculated orientation: [41.  0. 49.]
----------------------
Real: [  0   0 135]
Calculated orientation: [44.  0. 91.]
----------------------
Real: [  0   0 180]
Calculated orientation: [ 44.   0. 136.]
----------------------
Real: [ 0 45  0]
Calculated orientation: [  0.  45. 180.]
----------------------
Real: [ 0 45 45]
Calculated orientation: [ 0. 45. 45.]
----------------------
Real: [ 0 45 90]
Calculated orientation: [ 0. 45. 90.]
----------------------
Real: [  0  45 135]
Calculated orientation: [  0.  45. 135.]
----------------------
Real: [  0  45 180]
Calculated orientation: [  0.  45. 180.]
----------------------
Real: [ 0 90  0]
Calculated orientation: [ 0. 90.  0.]
----------------------
Real: [ 0 90 45]
Calculated orientation: [ 0. 90. 45.]
----------------------
Real: [ 0 90 90]
Calculated orientation:

In [8]:
for index, orientation in enumerate(database.euler):
    print('Real:', orientation)

    measures = np.column_stack((database.T_values[index],
                                database.azimuths[index],
                                np.full_like(database.azimuths[index], 90)))

    f.find_orientation_diffevol(measurements=measures, params=(90, 50, 20))
    print('----------------------')

Real: [0 0 0]
Calculated Orientation: [ 43. 180.  43.]
----------------------
Real: [ 0  0 45]
Calculated Orientation: [ 43. 180.  88.]
----------------------
Real: [ 0  0 90]
Calculated Orientation: [43.  0. 47.]
----------------------
Real: [  0   0 135]
Calculated Orientation: [45.  0. 90.]
----------------------
Real: [  0   0 180]
Calculated Orientation: [ 43. 180.  43.]
----------------------
Real: [ 0 45  0]
Calculated Orientation: [  0. 135.   0.]
----------------------
Real: [ 0 45 45]
Calculated Orientation: [ 0. 45. 45.]
----------------------
Real: [ 0 45 90]
Calculated Orientation: [ 0. 45. 90.]
----------------------
Real: [  0  45 135]
Calculated Orientation: [46. 20. 90.]
----------------------
Real: [  0  45 180]
Calculated Orientation: [  0. 135.   0.]
----------------------
Real: [ 0 90  0]
Calculated Orientation: [ 0. 90.  0.]
----------------------
Real: [ 0 90 45]
Calculated Orientation: [ 0. 90. 45.]
----------------------
Real: [ 0 90 90]
Calculated Orientation: